In [ ]:
import numpy as np
import h5py

CaIndicator = 'GP43'
Zoom = 'highzoom'
SessionId = 'data_141001_cell1_001'

In [ ]:
hf = h5py.File(CaIndicator + '_' + Zoom + '_nwb/' + SessionId + '.nwb', 'r')

In [ ]:
FMeanNeuropil = np.array(hf['acquisition']['timeseries']['FMeanNeuropil']['data'])
FMeanROI = np.array(hf['acquisition']['timeseries']['FMeanROI']['data'])
CaTimeStamps = np.array(hf['acquisition']['timeseries']['FMeanROI']['timestamps'])

SpkTimes = np.array(hf['processing']['Ephys']['UnitTimes']['SpikeTimes_0']['times'])

In [ ]:
FMean = FMeanROI - 0.7 * FMeanNeuropil
MaxF = FMean.max()
MaxTime = CaTimeStamps.max()

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline

fig, ax = plt.subplots()
ax.plot(CaTimeStamps, FMean, '-b', label='Raw fluorescence')
ax.plot(SpkTimes, np.ones(len(SpkTimes))*(MaxF+1), '+k', label='Spike times')
ax.legend()
ax.axis([0, MaxTime, 0, MaxF+5])
ax.set_xlabel('Time (s)')
ax.set_ylabel('F')
ax.set_title(CaIndicator + '_' + Zoom + '_' + SessionId)
plt.show()

In [ ]:
skip_interval = 3.0 # ignore F after a spike (< interval_pre sec) while computing baseline
spikePerCaBin = np.histogram(SpkTimes, bins=CaTimeStamps)[0]
spikePerCaBin = np.append([0], spikePerCaBin)

In [ ]:
from scipy.interpolate import interp1d

def get_baseline_corr_dff(fmean, spktimes, ca_time=CaTimeStamps, interval=skip_interval):
    fmeanSkipSpk = np.copy(fmean)
    for n_spk in spktimes:
        skip_frame = np.logical_and(ca_time >= n_spk, ca_time <= n_spk + interval)
        fmeanSkipSpk[skip_frame] = np.nan
    frameNoSpk = np.logical_not(np.isnan(fmeanSkipSpk))
    # baseline = interp1d(ca_time[frameNoSpk], fmean[frameNoSpk], kind='linear') 
    # 'cubic' interp1d behaves different in scipy from that in matlab, probably the num is different by default
    # f_baseline = baseline(ca_time)
    f_baseline = fmean[frameNoSpk].mean()
    dff = (fmean - f_baseline) / f_baseline
    return dff
    

In [ ]:
dff = get_baseline_corr_dff(FMean, SpkTimes)
MaxDFF = dff.max()

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline

fig, ax = plt.subplots()

ax.plot(CaTimeStamps, dff, '-b', label='dF/F')
ax.plot(SpkTimes, np.ones(len(SpkTimes))*(MaxDFF+.5), '+k', label='Spike times')
ax.legend()
ax.axis([0, MaxTime, -1, MaxDFF+1])
ax.set_xlabel('Time (s)')
ax.set_ylabel('dF/F')
ax.set_title(CaIndicator + '_' + Zoom + '_' + SessionId)
plt.show()